Training based on [this](https://colab.research.google.com/github/m3hrdadfi/soxan/blob/main/notebooks/Emotion_recognition_in_Greek_speech_using_Wav2Vec2.ipynb) notebook.

# Load Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !unzip -q "/content/drive/MyDrive/Colab_Notebooks/Sirius_ML/Project/ravdess/wav2vec_ravdess_model.zip"

# noises
!tar -xf /content/drive/MyDrive/Colab_Notebooks/Sirius_ML/ya_contest/speech_commands_data.tar.gz

In [3]:
!mkdir "ravdes"
!cp "/content/drive/MyDrive/Colab_Notebooks/Sirius_ML/Project/ravdess/Audio_Speech_Actors_01-24.zip" "ravdess_data.zip"
!unzip -q "/content/ravdess_data.zip" -d "/content/ravdes"

!cp "/content/drive/MyDrive/Colab_Notebooks/Sirius_ML/Project/ravdess/train_annotation.csv" "/content/ravdes"
!cp "/content/drive/MyDrive/Colab_Notebooks/Sirius_ML/Project/ravdess/test_annotation.csv" "/content/ravdes"

test_annotation_path = "/content/ravdes/test_annotation.csv"
train_annotation_path = "/content/ravdes/train_annotation.csv"

data_path = "/content/ravdes/"

# Emotion Recognition Using Wav2Vec 2.0

In [4]:
%%capture

!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install jiwer
!pip install torchaudio
!pip install librosa


In [ ]:
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/content/cache
%env HF_DATASETS_CACHE=/content/cache
%env CUDA_LAUNCH_BLOCKING=1

env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=/content/cache
env: HF_DATASETS_CACHE=/content/cache
env: CUDA_LAUNCH_BLOCKING=1


## Prepare Data



In [5]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm.notebook import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

import os
import sys

In [6]:
train_df = pd.read_csv(train_annotation_path).drop("Unnamed: 0", axis=1)
test_df = pd.read_csv(test_annotation_path).drop("Unnamed: 0", axis=1)

emo_dict = {1: "neutral", 2: "calm", 3: "happy", 4: "sad", 5: "angry", 6: "fearful", 7: "disgust", 8: "surprised"}

for df in [train_df, test_df]:
    df['emotion_word'] = df['emotion'].apply(lambda x: emo_dict[int(x)])
    df['path'] = data_path + df['file_path']

train_df.head()

,Unnamed: 0.1,file_name,actor,emotion,intensity,file_path,emotion_word,path
0,1,03-01-03-02-02-01-09.wav,9,3,2,Actor_09/03-01-03-02-02-01-09.wav,happy,/content/ravdes/Actor_09/03-01-03-02-02-01-09.wav
1,2,03-01-02-01-01-02-09.wav,9,2,1,Actor_09/03-01-02-01-01-02-09.wav,calm,/content/ravdes/Actor_09/03-01-02-01-01-02-09.wav
2,4,03-01-05-01-01-02-09.wav,9,5,1,Actor_09/03-01-05-01-01-02-09.wav,angry,/content/ravdes/Actor_09/03-01-05-01-01-02-09.wav
3,5,03-01-05-02-01-01-09.wav,9,5,2,Actor_09/03-01-05-02-01-01-09.wav,angry,/content/ravdes/Actor_09/03-01-05-02-01-01-09.wav
4,6,03-01-04-02-01-02-09.wav,9,4,2,Actor_09/03-01-04-02-01-02-09.wav,sad,/content/ravdes/Actor_09/03-01-04-02-01-02-09.wav


Check if files are correct.

In [7]:
def check_files(df):
    for ind, path in tqdm(zip(df.index, df['path'])):
        try:
            a, s = torchaudio.load(path)
            if len(a) == 0:
                raise Exception('null_len')
        except Exception as e:
            print(f"On {path} found: \n{e}")
            df = df.drop(ind, axis=0)

    return df.reset_index(drop=True)

train_df = check_files(train_df)
print(len(test_df))
test_df = check_files(test_df)
print(len(test_df))

0it [00:00, ?it/s]

427


0it [00:00, ?it/s]

427


In [ ]:
train_df.index

RangeIndex(start=0, stop=1074, step=1)

In [8]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

idx = np.random.randint(0, len(df))
sample = df.iloc[idx]
path = sample["path"]
label = sample["emotion"]


print(f"ID Location: {idx}")
print(f"      Label: {label}")
print()

speech, sr = torchaudio.load(path)
speech = speech[0].numpy().squeeze()
speech = librosa.resample(np.asarray(speech), sr, 16000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

ID Location: 130
      Label: 3



In [9]:
train_df.to_csv(data_path + '/train.csv')
test_df.to_csv(data_path + '/test.csv')

## Prepare Data for Training

In [10]:
# Loading the created dataset using datasets
from datasets import load_dataset, load_metric


data_files = {
    "train": data_path + '/train.csv', 
    "validation": data_path + '/test.csv',
}

dataset = load_dataset("csv", data_files=data_files, delimiter=",", )
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

print(train_dataset)
print(eval_dataset)

Using custom data configuration default-fdb61f0e29dffa98


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-fdb61f0e29dffa98/0.0.0/caa45397026a471644fd1f91ff50753ed056ba9dc97d0345dbeebe8f4ed51833. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['Unnamed: 0', 'Unnamed: 0.1', 'file_name', 'actor', 'emotion', 'intensity', 'file_path', 'emotion_word', 'path'],
    num_rows: 1074
})
Dataset({
    features: ['Unnamed: 0', 'Unnamed: 0.1', 'file_name', 'actor', 'emotion', 'intensity', 'file_path', 'emotion_word', 'path'],
    num_rows: 427
})


In [11]:
# We need to specify the input and output column
input_column = "path"
output_column = "emotion_word"

In [12]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 8 classes: ['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']


In [13]:
from transformers import AutoConfig, Wav2Vec2Processor

In [14]:
model_name_or_path = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
pooling_mode = "mean"

In [15]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [16]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

Downloading:   0%|          | 0.00/214 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The target sampling rate: 16000


/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


# Preprocess Data

In [17]:
def speech_file_to_array_fn(path):
    speech, sampling_rate = librosa.load(path, target_sampling_rate)
    return speech

def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]

    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

In [18]:
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=100,
    # num_proc=4
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    # num_proc=4
)

  0%|          | 0/11 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


  0%|          | 0/5 [00:00<?, ?ba/s]

In [19]:
idx = 3
print(f"Training input_values: {train_dataset[idx]['input_values']}")
print(f"Training attention_mask: {train_dataset[idx]['attention_mask']}")
print(f"Training labels: {train_dataset[idx]['labels']} - {train_dataset[idx]['emotion_word']}")

Training input_values: [0.00013706796744372696, 0.00029290758538991213, 7.609635758853983e-06, 0.0002186823112424463, 0.00021206891688052565, 0.0002141365985153243, 0.00036366473068483174, 0.00017260915774386376, 0.00011738135071936995, 0.0003807850298471749, 0.00019560665532480925, 0.00035456896875984967, 0.0004870143311563879, 0.0004268776683602482, 0.0006281686946749687, 0.0005395019543357193, 0.0005629059160128236, 0.0005194583209231496, 0.0006504361517727375, 0.0006424871389754117, 0.000859964988194406, 0.000757167290430516, 0.0008310553967021406, 0.0007566031417809427, 0.0007849781541153789, 0.0011193944374099374, 0.001081740134395659, 0.0010058940388262272, 0.001076629851013422, 0.0010475557064637542, 0.0008708674577064812, 0.001054835389368236, 0.0007202692213468254, 0.0007223269785754383, 0.0008759927586652339, 0.0007655834197066724, 0.0006765604484826326, 0.0008435205672867596, 0.0008180300937965512, 0.0006909592193551362, 0.0006390805938281119, 0.000616588513366878, 0.000683

In [20]:
emo_dict = dict()

for idx in range(100):
    if len(emo_dict.keys()) == 8:
        break
    emo_dict[train_dataset[idx]['labels']] = train_dataset[idx]['emotion_word']

emo_dict

{0: 'angry',
 1: 'calm',
 2: 'disgust',
 3: 'fearful',
 4: 'happy',
 5: 'neutral',
 6: 'sad',
 7: 'surprised'}

Great, now we've successfully read all the audio files, resampled the audio files to 16kHz, and mapped each audio to the corresponding label.

## Model

Before diving into the training part, we need to build our classification model based on the merge strategy. 

In [21]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


In [22]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)


class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    # def __init__(self, config):
    #     super().__init__()
    #     self.dense = nn.Linear(config.hidden_size, config.hidden_size // 2)
    #     self.dropout = nn.Dropout(config.final_dropout)
    #     self.out_proj = nn.Linear(config.hidden_size // 2, config.num_labels)

    # def forward(self, features, **kwargs):
    #     x = features
    #     x = self.dropout(x)
    #     x = self.dense(x)
    #     x = torch.tanh(x)
    #     x = self.dropout(x)
    #     x = self.out_proj(x)
    #     return x

    def __init__(self, config):
        super().__init__()
        self.dense1 = nn.Linear(config.hidden_size, config.hidden_size // 2)
        self.dropout1 = nn.Dropout(.25)
        self.dense2 = nn.Linear(config.hidden_size // 2, config.hidden_size // 4)
        self.dropout2 = nn.Dropout(.25)
        self.out_proj = nn.Linear(config.hidden_size // 4, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout1(x)
        x = self.dense1(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.dense2(x)
        x = torch.relu(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of XLSR-Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [23]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None
    noises: List[List[float]] = None
    
    # The model may not start learning right away if the classification head 
    # is too large or if there is a lot of data (for example, if you add noise). 
    # You can first try to teach without noise and with a small head, and then finish teaching. 

    def make_some_noise(self, clean):
        clean = np.array(clean)
        max_amp = 1 - np.random.random() / 3
        # max_amp = 1
        noise = noises[np.random.randint(0, len(noises))]
        noise_amp = np.random.rand() * max_amp
        max_start = len(noise) - len(clean)
        start = np.random.randint(0, max_start + 1)
        noise_part = noise[start:start+len(clean)]
        noise_mult = np.abs(clean.max()) / np.abs(noise_part).max() * noise_amp
        return (clean + noise_part * noise_mult) / (1 + noise_amp)

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": self.make_some_noise(feature["input_values"])} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

You can find noises in some kaggle competition. 

In [24]:
noises = []
for noise_name in os.listdir("./noises"):
    path = os.path.join("./noises", noise_name)
    noises.append(torchaudio.load(path)[0].squeeze().numpy())

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True, noises=noises)

Next, the evaluation metric is defined. There are many pre-defined metrics for classification/regression problems, but in this case, we would continue with just **Accuracy** for classification and **MSE** for regression. You can define other metrics on your own.

In [25]:
is_regression = False

In [26]:
import numpy as np
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

Now, we can load the pretrained XLSR-Wav2Vec2 checkpoint into our classification model with a pooling strategy.

In [27]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english were not used when initializing Wav2Vec2ForSpeechClassification: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['classifier.dense1.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense1.bias', 'classifier.dense2.weight', 'clas

The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [ ]:
# model.freeze_feature_extractor()

In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

**Note**: If one wants to save the trained models in his/her google drive the commented-out `output_dir` can be used instead.

There should be at least 20 epochs to get accuracy >70 if you use noises.

In [28]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/wav2vec2-xlsr-sirius",
    # output_dir="/content/gdrive/MyDrive/wav2vec2-xlsr-greek-speech-emotion-recognition"
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=15.0,
    fp16=True,
    save_steps=100,
    eval_steps=20,
    logging_steps=10,
    logging_dir="./logs",
    learning_rate=1e-4,
    save_total_limit=2,
)

For future use we can create our training script, we do it in a simple way. You can add more on you own.

In [29]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.

        Args:
            model (:obj:`nn.Module`):
                The model to train.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument :obj:`labels`. Check your model's documentation for all accepted arguments.

        Return:
            :obj:`torch.Tensor`: The tensor with training loss on this batch.
        """

        model.train()
        inputs = self._prepare_inputs(inputs)

        if self.use_amp:
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_amp:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()


Now, all instances can be passed to Trainer and we are ready to start training!

In [30]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
)

Using amp fp16 backend


### Training

Training will take between 10 and 60 minutes depending on the GPU allocated to this notebook. 

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (right mouse click -> inspect -> Console tab and insert code).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [ ]:
!rm -r "/content/wav2vec2-xlsr-sitius/checkpoint-50"

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: emotion, actor, file_path, intensity, Unnamed: 0.1, Unnamed: 0, emotion_word, path, file_name.
***** Running training *****
  Num examples = 1074
  Num Epochs = 15
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 2010
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return t

Step,Training Loss,Validation Loss,Accuracy
20,2.078500,2.079696,0.131148
40,2.081700,2.075857,0.138173
60,2.072500,2.068946,0.133489
80,2.053800,2.064727,0.121780
100,2.072200,2.060743,0.133489
120,2.058800,2.060315,0.133489
140,2.197300,2.072524,0.140515
160,2.068800,2.070061,0.133489
180,2.058900,2.065588,0.133489
200,2.071400,2.062849,0.133489


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: emotion, actor, file_path, intensity, Unnamed: 0.1, Unnamed: 0, emotion_word, path, file_name.
***** Running Evaluation *****
  Num examples = 427
  Batch size = 4
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: emotion, actor, file_path, intensity, Unnamed: 0.1, Unnamed: 0, emotion_word, path, file_name.
***** Running Evaluation *****
  Num examples = 427
  Batch size = 4
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: emotion, actor, file_path, intensity, Unnamed: 0.1, Unnamed: 0, emotion_word, path, file_name.
***** Running Evaluation *****
  Num examples = 427
  Batch size = 4
The following columns in the evaluation set  don't have a cor

In [ ]:
!zip -r -0 /content/drive/MyDrive/Colab_Notebooks/Sirius_ML/Project/w2v_git.zip /content/wav2vec2-xlsr-sirius/checkpoint-2000

  adding: content/wav2vec2-xlsr-sitius/ (stored 0%)
  adding: content/wav2vec2-xlsr-sitius/runs/ (stored 0%)
  adding: content/wav2vec2-xlsr-sitius/runs/Jul13_20-02-47_831b64cc40b8/ (stored 0%)
  adding: content/wav2vec2-xlsr-sitius/runs/Jul13_20-02-47_831b64cc40b8/1626206643.9698489/ (stored 0%)
  adding: content/wav2vec2-xlsr-sitius/runs/Jul13_20-02-47_831b64cc40b8/1626206643.9698489/events.out.tfevents.1626206643.831b64cc40b8.62.9 (deflated 62%)
  adding: content/wav2vec2-xlsr-sitius/runs/Jul13_20-02-47_831b64cc40b8/events.out.tfevents.1626206643.831b64cc40b8.62.8 (deflated 68%)
  adding: content/wav2vec2-xlsr-sitius/runs/Jul13_20-01-18_831b64cc40b8/ (stored 0%)
  adding: content/wav2vec2-xlsr-sitius/runs/Jul13_20-01-18_831b64cc40b8/1626206491.9228146/ (stored 0%)
  adding: content/wav2vec2-xlsr-sitius/runs/Jul13_20-01-18_831b64cc40b8/1626206491.9228146/events.out.tfevents.1626206491.831b64cc40b8.62.5 (deflated 62%)
  adding: content/wav2vec2-xlsr-sitius/runs/Jul13_20-01-18_831b64cc

In [ ]:
!cp "/content/wav2vec_ravdess_model.zip" "/content/drive/MyDrive/Colab_Notebooks/Sirius_ML/Project/ravdess"

The training loss goes down and we can see that the Acurracy on the test set also improves nicely. Because this notebook is just for demonstration purposes, we can stop here.

The resulting model of this notebook has been saved to [m3hrdadfi/wav2vec2-xlsr-greek-speech-emotion-recognition](https://huggingface.co/m3hrdadfi/wav2vec2-xlsr-greek-speech-emotion-recognition)

As a final check, let's load the model and verify that it indeed has learned to recognize the emotion in the speech.

Let's first load the pretrained checkpoint.

## New Data prep

In [ ]:
# !unzip -q "new_data.zip" -d "new_data"

In [ ]:
path = "new_data/new_data/"
new_df = {}
new_df['path'] = []
new_df['emotion_word'] = []
for filename in os.listdir(path):
    new_df['emotion_word'].append(filename.split("_")[-2])
    new_df['path'].append(path + filename)

new_df = pd.DataFrame.from_dict(new_df)
new_df.to_csv("new_data/new_df.csv")

## Evaluation

In [ ]:
import librosa
from sklearn.metrics import classification_report

In [ ]:
new_df_path = "/content/new_data/new_df.csv"
test_dataset = load_dataset("csv", data_files={"test": new_df_path}, delimiter=",")["test"]
test_dataset

Using custom data configuration default-f8da6bbd557619d6


Dataset csv downloaded and prepared to /content/cache/csv/default-f8da6bbd557619d6/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


Dataset({
    features: ['Unnamed: 0', 'path', 'emotion_word'],
    num_rows: 48
})

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


In [ ]:
model_name_or_path = "/content/content/wav2vec2-xlsr-sitius/checkpoint-1206"
config = AutoConfig.from_pretrained(model_name_or_path) # model from checkpoint or huggingface
processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english") # model for processor only from hugginface
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device) # model from checkpoint or huggingface

In [ ]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch


def predict(batch):
    features = processor(batch["speech"], sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits 

    pred_ids = torch.argmax(logits, dim=-1).detach().cpu().numpy()
    batch["predicted"] = pred_ids
    return batch

In [ ]:
test_dataset = test_dataset.map(speech_file_to_array_fn)

In [ ]:
result = test_dataset.map(predict, batched=True, batch_size=8)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [ ]:
label_names = [config.id2label[i] for i in range(config.num_labels)]
label_names

['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']

In [ ]:
y_true = [config.label2id[name] for name in result["emotion_word"]]
y_pred = result["predicted"]

print(y_true[:5])
print(y_pred[:5])

[3, 3, 4, 1, 5]
[6, 6, 1, 6, 1]


In [ ]:
print(classification_report(y_true, y_pred, target_names=label_names))

              precision    recall  f1-score   support

       angry       0.75      0.33      0.46         9
        calm       0.10      0.20      0.13         5
     disgust       0.33      0.75      0.46         4
     fearful       1.00      0.17      0.29         6
       happy       1.00      0.12      0.22         8
     neutral       0.00      0.00      0.00         7
         sad       0.10      0.50      0.17         4
   surprised       0.67      0.40      0.50         5

    accuracy                           0.27        48
   macro avg       0.49      0.31      0.28        48
weighted avg       0.55      0.27      0.28        48



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Prediction

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from transformers import AutoConfig, Wav2Vec2Processor

import librosa
import IPython.display as ipd
import numpy as np
import pandas as pd

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name_or_path = "/content/content/wav2vec2-xlsr-sitius/checkpoint-1206"
config = AutoConfig.from_pretrained(model_name_or_path)
processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
sampling_rate = processor.feature_extractor.sampling_rate
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

In [ ]:
def speech_file_to_array_fn(path, sampling_rate):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech


def predict(speech, sampling_rate):
    # speech = speech_file_to_array_fn(path, sampling_rate)
    features = processor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Emotion": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs


STYLES = """
<style>
div.display_data {
    margin: 0 auto;
    max-width: 500px;
}
table.xxx {
    margin: 50px !important;
    float: right !important;
    clear: both !important;
}
table.xxx td {
    min-width: 300px !important;
    text-align: center !important;
}
</style>
""".strip()

def prediction(df_row):
    path, emotion = df_row["path"], df_row["emotion_word"]
    df = pd.DataFrame([{"Emotion": emotion, "Sentence": "    "}])
    setup = {
        'border': 2,
        'show_dimensions': True,
        'justify': 'center',
        'classes': 'xxx',
        'escape': False,
    }
    ipd.display(ipd.HTML(STYLES + df.to_html(**setup) + "<br />"))
    speech, sr = torchaudio.load(path)
    speech = speech[0].numpy().squeeze()
    speech = librosa.resample(np.asarray(speech), sr, sampling_rate)
    ipd.display(ipd.Audio(data=np.asarray(speech), autoplay=True, rate=sampling_rate))

    outputs = predict(path, sampling_rate)
    r = pd.DataFrame(outputs)
    ipd.display(ipd.HTML(STYLES + r.to_html(**setup) + "<br />"))

In [ ]:
speech, sr = librosa.load("/content/111415597_427_sad_.wav", sr=16000)
dist = predict(speech, 16000)

In [ ]:
[f"{d['Emotion']} - {d['Score']}" for d in dist]

['angry - 0.0%',
 'calm - 99.9%',
 'disgust - 0.0%',
 'fearful - 0.0%',
 'happy - 0.0%',
 'neutral - 0.0%',
 'sad - 0.1%',
 'surprised - 0.0%']

In [ ]:
test = pd.read_csv("/content/new_data/new_df.csv", sep=",")
test.head()

,Unnamed: 0,path,emotion_word
0,0,new_data/new_data/111415597_455_fearful_.wav,fearful
1,1,new_data/new_data/255547109_586_fearful_.wav,fearful
2,2,new_data/new_data/415713765_505_happy_.wav,happy
3,3,new_data/new_data/111415597_451_calm_.wav,calm
4,4,new_data/new_data/787369606_447_neutral_.wav,neutral


In [ ]:
gen = iter(range(len(test)))

In [ ]:
# ind = np.random.randint(0, len(test))
prediction(test.iloc[next(gen)])

,Emotion,Sentence
0,sad,


,Emotion,Score
0,angry,0.0%
1,calm,0.0%
2,disgust,0.1%
3,fearful,0.0%
4,happy,0.0%
5,neutral,0.0%
6,sad,99.9%
7,surprised,0.0%


In [ ]:
track, sr = librosa.load("/content/111415597_427_sad_.wav", sr=16000)
ipd.display(ipd.Audio(data=np.asarray(track), autoplay=True, rate=16000))
# pip install noisereduce

In [ ]:
import noisereduce as nr
clip = track
rtrack = nr.reduce_noise(track, clip)

In [ ]:
ipd.display(ipd.Audio(data=np.asarray(rtrack), autoplay=True, rate=16000))